In [1]:
import arff, numpy as np
import pandas as pd 
dataset = arff.load(open('vote.arff', 'rb'))
data = np.array(dataset['data'])
data_df = pd.DataFrame(data)
data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,n,y,n,y,y,y,n,n,n,y,None,y,y,y,n,y,republican
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,None,republican
2,None,y,y,None,y,y,n,n,n,n,y,n,y,y,n,n,democrat
3,n,y,y,n,None,y,n,n,n,n,y,n,y,n,n,y,democrat
4,y,y,y,n,y,y,n,n,n,n,y,None,y,y,y,y,democrat


In [16]:
data_df.fillna(value='?', inplace=True)
data_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y,republican
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?,republican
2,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n,democrat
3,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y,democrat
4,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y,democrat


In [17]:
from sklearn.model_selection import KFold
from __future__ import division
N_SPLITS = 10
len_data = len(data_df)
accuracy_mean= 0

#K-fold with 10 splits 
kf = KFold(n_splits=N_SPLITS, shuffle = True)


for train, test in kf.split(data_df):
    train_data = np.array(data_df)[train]
    test_data = np.array(data_df)[test]
    votes_democrat = {"y": [0] * 16, "n": [0] * 16, "?": [0] * 16}
    votes_republican = {"y": [0] * 16, "n": [0] * 16, "?": [0] * 16}
    n_democrats = 0
    n_republicans = 0

    for example in train_data:
        current = {"y": [0] * 16, "n": [0] * 16, "?": [0] * 16}
        if example[16] == "democrat":
            n_democrats += 1
            current = votes_democrat
        else:
            n_republicans += 1
            current = votes_republican
        for i in range(1, len(example)):
            current[example[i-1]][i-1] += 1
        #print(current)

#Classification

    classification = []
    for example in test_data:
        p_democrat = 1
        for i in range(1, len(example)):
            p_democrat *= votes_democrat[example[i-1]][i-1] / n_democrats
        p_democrat *= n_democrats / len_data

        p_republican = 1
        for i in range(1, len(example)):
            p_republican *= votes_republican[example[i-1]][i-1] / n_republicans
        p_republican *= n_republicans / len_data
        #print(p_democrat, p_republican)
        category = ""
        if p_democrat == p_republican:
            category = (random.choice(["democrat", "republican"]), p_democrat)
        else:
            category = max(("democrat", p_democrat), ("republican", p_republican), key=lambda prob: prob[1])
        classification.append(category)

    accuracy = 0
    for i in range(0, len(test_data)):
        if classification[i][0] == test_data[i][16]:
            accuracy += 1
    accuracy /= len(test_data)
    accuracy_mean += accuracy
    print("Current accuracy is {0}%".format(accuracy * 100))
accuracy_mean /= N_SPLITS
print("Mean accuracy is {0}%".format(accuracy_mean*100))



Current accuracy is 95.4545454545%
Current accuracy is 90.9090909091%
Current accuracy is 86.3636363636%
Current accuracy is 93.1818181818%
Current accuracy is 86.3636363636%
Current accuracy is 90.6976744186%
Current accuracy is 95.3488372093%
Current accuracy is 81.3953488372%
Current accuracy is 95.3488372093%
Current accuracy is 90.6976744186%
Mean accuracy is 90.5761099366%
